### BME Lab 111 - High-Resolution Computer Tomography in the Clinic for Osteoporosis

#### Day 3 - hFE post-processing and analysis

Date: 26.02.2024

Author : Simone Poncioni (MSB - ARTORG Center)

**Step 1: prepare the workspace by importing the necessary libraries and loading the raw data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.interpolate import interp1d

In [ ]:
# TODO: insert here the path to the .csv file containing the force-displacement curve
hfe_path = Path('insert/filepath/here.csv')

In [ ]:
# import raw data
hfe_df = pd.read_csv(hfe_path)

# store columns in variables --> mind the spaces in front of the column names
displacement = hfe_df['   displacement']
force = hfe_df['          force']

# check the data in a simple scatter plot
plt.figure(figsize=(6, 5))
plt.scatter(displacement, force, label=hfe_path.stem)
plt.title(f'Raw results for {hfe_path.stem}', weight='bold')
plt.xlabel('Displacement [mm]')
plt.ylabel('Force [N]')
plt.legend()
plt.show()

Considerations:

- Under SI units, the displacement and reaction forces are negative because we are considering a **compressive** loading condition.
- The material behaves linearly before yielding, and post-yield there is damage accumulation that leads to non-reversible plasticity.

**Step 2: Calculate basic mechanical properties**

In [ ]:
# TODO: redefine variables for the specific test
LINEAR_REG_IDX =  # index in the displacement array for which the curve is still linear (-) --> count the "dots" in above plot
CROSS_SECTION =  # (mm2) => Tt.Ar from Standard Evaluation
NB_SLICES =  # (-) => number of slices in Standard Evaluation (changes depending on radius/tibia)

In [ ]:
# For simplicity, let's put the data in absolute values
displacement_abs = np.abs(displacement.values)
force_abs = np.abs(force.values)

Calculate the stiffness of the linear region of the curve using the formula:

$K = \frac{F}{d} \left( \frac{N}{mm} \right)$

In [ ]:
def calculate_stiffness(force: np.ndarray, displacement: np.ndarray, lin: int):
    """
    Calculate the stiffness of a system given force and displacement.

    This function calculates the stiffness (k) of a system by fitting a 
    first order polynomial to the linear portion of the force-displacement data.

    Args:
        force (np.ndarray): The force applied to the system.
        displacement (np.ndarray): The displacement of the system under the applied force.
        lin (int): The index up to which the data is considered linear.

    Returns:
        float: The stiffness of the system.

    Prints:
        str: The calculated stiffness value.
    """
    disp_linear = displacement[0:lin]
    force_linear = force[0:lin]
    # TODO: calculate k (the stiffness) using a 1st order polynomial fit (https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html)
    
    print(f'Calculated stiffness k:\t{k[0]:.3f} (N/mm)')
    return k[0]

k = calculate_stiffness(force_abs, displacement_abs, LINEAR_REG_IDX)

In [ ]:
def calculate_fmax_and_disp_at_fmax(force: np.ndarray, displacement: np.ndarray):
    """
    Calculate the maximum force and the displacement at the maximum force.

    Args:
        force (np.ndarray): An array of force values.
        displacement (np.ndarray): An array of displacement values.

    Returns:
        tuple: A tuple containing the maximum force, the displacement at the maximum force, and the index of the maximum force.
    """
    def calculate_fmax(force: np.ndarray):
        """
        Calculate the maximum force from the force array.

        Args:
            force (np.ndarray): An array of force values.

        Returns:
            float: The maximum force.
        """
        # TODO: calculate the maximum force (fmax) from the force array
        # https://numpy.org/doc/stable/reference/generated/numpy.max.html
        fmax = 
        return fmax
    
    fmax = calculate_fmax(force)
    idx_fmax = np.where(force == fmax)
    return fmax, displacement[idx_fmax[0][0]], idx_fmax


def yield_point(height, k, FZ, DZ):
    """
    Calculate the yield point.

    Args:
        height (float): The sample height.
        k (float): The stiffness.
        FZ (np.ndarray): The reaction force in Z direction.
        DZ (np.ndarray): The displacement in Z direction.

    Returns:
        tuple: A tuple containing the yield force and the displacement at the yield point.
    """
    h02 = height * 0.002
    intersect02 = -h02 * k
    F_offsetrule = np.array(DZ) * k + intersect02
    for i in range(1, len(FZ)):
        if FZ[i] < F_offsetrule[i]:
            s_temp = (FZ[i] - FZ[i - 1]) / (DZ[i] - DZ[i - 1])
            int_temp = FZ[i - 1] - s_temp * DZ[i - 1]
            disp_yield = (int_temp - intersect02) / (k - s_temp)
            Fyield = s_temp * disp_yield + int_temp
            break
    print("    - F yield = ", str(Fyield), " N")
    print("    - Displacement at F yield = ", str(disp_yield), " mm")
    return Fyield, disp_yield

In [ ]:
spacing = 0.061
height_sample_mm = NB_SLICES * spacing # mm

print(f"Sample height: {height_sample_mm} mm")
print(f"Cross section: {CROSS_SECTION} mm²")

In [ ]:
# Run previously written functions
k = calculate_stiffness(force_abs, displacement_abs, lin=LINEAR_REG_IDX)
fmax, disp_at_fmax, idx_fmax = calculate_fmax_and_disp_at_fmax(force_abs, displacement_abs)
force_yield, displacement_yield = yield_point(height=height_sample_mm, k=k, FZ=force_abs, DZ=displacement_abs)

Calculate the stress using the following formula:

$\sigma = \frac{F}{A} \left(\frac{N}{mm^2}\right)$

Where:

$F: \text{reaction force}$

$A: \text{cross-sectional area}$

In [ ]:
def stress(force, force_yield, CROSS_SECTION):
    """
    Calculate the stress, maximum stress, and yield stress.

    Args:
        force (np.ndarray): An array of force values.
        force_yield (float): The yield force.
        CROSS_SECTION (float): The cross-sectional area.

    Returns:
        tuple: A tuple containing the stress, maximum stress, and yield stress.
    """
    # TODO: calculate sigma, sigma_max and sigma_yield from the forces and the cross section
    sigma =
    sigma_max = 
    sigma_yield = 
    return sigma, sigma_max, sigma_yield

sigma, sigma_max, sigma_yield = stress(force_abs, force_yield, CROSS_SECTION)

Calculate the strain using the following formula:

$\epsilon = \frac{\Delta l}{l_0} \left(\frac{mm}{mm}\right) (-)$

Where:

$\Delta l: \text{displacement}$

$l_0: \text{sample heigth}$

In [ ]:
def strain(displacement: np.ndarray, displacement_yield: float, disp_at_fmax: float, height: float):
    """
    Calculate the strain, yield strain, and maximum strain.

    Args:
        displacement (np.ndarray): An array of displacement values.
        displacement_yield (float): The yield displacement.
        disp_at_fmax (float): The displacement at the maximum force (fmax).
        height (float): The initial height of the sample.

    Returns:
        tuple: A tuple containing the strain, yield strain, and maximum strain.
    """
    # TODO: calculate the strain, the yield strain and the maximum strain
    return epsilon, eps_yield, eps_sigma_max

epsilon, eps_yield, eps_sigma_max = strain(displacement_abs, displacement_yield, disp_at_fmax, height_sample_mm)
stress_strain = np.c_[epsilon, sigma]

In [ ]:
# Plot the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
force_displacement = np.insert(np.c_[displacement_abs, force_abs], 0, [0, 0], axis=0)
ax1.plot(force_displacement[:, 0], force_displacement[:, 1])
ax1.plot([0.002 * height_sample_mm, displacement_yield], [0, force_yield], c='k', linestyle='--')
ax1.scatter(displacement_yield, force_yield, c='g', label='Yield force')
ax1.scatter(force_displacement[idx_fmax, 0], force_displacement[idx_fmax, 1], c='r', label='Failure force')
ax1.set_title('Force - Displacement', weight='bold')
ax1.set_xlabel('Displacement [mm]')
ax1.set_ylabel('Force [N]')
ax1.set_xlim(0)
ax1.set_ylim(0)
ax1.legend(fontsize=12)

stress_strain = np.insert(stress_strain, 0, [0, 0], axis=0)
ax2.plot(stress_strain[:, 0], stress_strain[:, 1])
ax2.plot([0.002, eps_yield], [0, sigma_yield], c='k', linestyle='--')
ax2.scatter(eps_yield, sigma_yield, c='g', label='yield stress')
ax2.scatter(eps_sigma_max, sigma_max, c='r', label='failure stress')
ax2.set_title('Stress - Strain', weight='bold')
ax2.set_xlabel('$\mathbf{\epsilon}$ [-]')
ax2.set_ylabel('$\mathbf{\sigma}$ [N/mm²]')
ax2.set_xlim(0)
ax2.set_ylim(0)
ax2.legend(fontsize=12)
fig.set_facecolor("white")

savefig_path = hfe_path.parent / f'{hfe_path.stem}_results.png'
fig.savefig(savefig_path, dpi=150, bbox_inches='tight')